* Repetir o exercício 2
* Eliminar o transiente usando a heurística MSER-5Y
* Analisar o estado estacionário usando o método Batch Means.
* Encontrar o intervalo de confiança de 95% para os seguintes casos
** Caso I
    - taxa de entrada: λ=9,0 clientes por segundo.
    - taxa de serviço: μ=10 clientes por segundo.
** Caso II
   - taxa de entrada: λ=9,5 clientes por segundo.
   - taxa de serviço: μ=10 clientes por segundo.

In [1]:
import random                      
import simpy # biblioteca de simulação
import math as mt
from tqdm import tqdm
import statistics as ss
import scipy.stats as st
import matplotlib.pyplot as plt
import numpy as np

In [2]:
taxaEntrada = 9           # taxa de entrada: λ=9 clientes por segundo. 
taxaServico = 10          # taxa de serviço: μ=10 clientes por segundo.

In [3]:
def media(lista):
    soma = 0
    for i in range(len(lista)):
        soma += lista[i]
    return soma/len(lista)

def varAmostral(lista):
    m = media(lista)
    soma = 0
    for i in range(len(lista)):
        soma += (lista[i]-m)**2
    return soma/(len(lista)-1)
      
def desPadrao(lista):
    dp = mt.sqrt(varAmostral(lista))
    return dp

def exponencial(b):
    u = random.random()
    x = -b * mt.log(1-u)

    return(x)

In [4]:
def tempoQuantiMedia(quanti, media):
    return [-(1/media) *  np.log(1 - np.random.uniform(0,1)) for _ in range(quanti-1)]

In [5]:
def mm1(n, lamb, mi):
    medioEsperaFila = tempoQuantiMedia(n, lamb)
    servicos = tempoQuantiMedia(n, mi)
    filaEsperas = [0]
    espera = 0
    i = 0
    while i < len(medioEsperaFila):
        servicoCompleto = servicos[i] + filaEsperas[-1]
        if (medioEsperaFila[i] <= servicoCompleto):
             espera = (servicos[i] - medioEsperaFila[i]) + filaEsperas[-1]
        else:
             espera = 0
        filaEsperas.append(espera)
        i+=1
    return filaEsperas

In [6]:
def mser5y(nClientes, x):
    auxList = []
    aux = 0
    z = []
    i = j =0
    while i < nClientes:
        while j < 5:
            auxList.append(x[i])
            i +=1
            j +=1
        z.append(media(auxList))
        auxList.clear()
        j=0
    mser5yList = []
    k = nClientes/5
    meio = k/2
    d = 0
    while d < meio:
        desvpZ = ss.fmean(z)
        aux = desvpZ / mt.sqrt(k-d)
        mser5yList.append(aux)
        d += 1
        del z[0]
    menor = min(mser5yList)              
    posicao = mser5yList.index(menor)
    return posicao

In [7]:
def batchmeans (lista, b, m): #funcao: lista dos elementos no estado estacionário, b = blocos, m = tamanho dos blocos
    n = b*m         #total de observações
    i = 0
    j = 0
    listaux = []
    listaMedias = []
    while i < n:
        while j < m:
            listaux.append(lista[i])
            i += 1
            j += 1
        aux = ss.mean(listaux)
        listaMedias.append(aux)     #lista com as médias dos blocos
        j = 0
        listaux.clear()
    return(listaMedias)

In [8]:
def ranking (lista):    #cria lista de rankings 
    lranking = []
    i = 0
    l = 0
    soma = 0
    while l < len(lista):
        while i < len(lista):
            if lista[i] <= lista[l]:
                soma += 1
            i += 1
        lranking.append(soma)
        soma = 0
        i = 0
        l += 1
    return(lranking)


In [9]:
def estatisticaVN(lista):
    tf = (len(lista) - 1) #B-1
    i = 0
    somatorio1 = 0
    somatorio2 = 0

    while i < tf:
        sub = lista[i] - lista[i+1]
        somatorio1 += mt.pow(sub, 2)
        i += 1

    mediaR = ss.mean(lista)
    i = 0
    while i < len(lista): #B
        sub = lista[i] - mediaR
        somatorio2 += mt.pow(sub, 2)
        i += 1 
    rvn = somatorio1 / somatorio2
    return(rvn)

In [10]:
def sts (lista, b, m):
    n = b*m
    i = 0
    j = 0
    listaux = []
    listaA = []

    while i < n:
        while j < m:
            aux = lista[i]
            mult = ((m + 1)/2) - (j+1)
            listaux.append(mult*aux)
            i += 1
            j += 1
        
        soma = sum(listaux)
        listaA.append(soma) 
        j = 0
        listaux.clear()

    return(listaA)

In [11]:
def E(taxaEntrada, taxaServico):
    p = taxaEntrada/taxaServico
    e = p*((1/taxaServico)/(1-p))
    return e

In [35]:
def simulacao(n, lamb, mi):
    i =0
    b = 20
    m = 100
    x = mm1(n, lamb, mi) # 10^5
    mser5yList =  mser5y(n, x)

    while mser5yList == 0:
        print('Ainda não se encontrou o ponto de truncagem.\n')
        print('A coletar mais informações até que se encontre...')
        n += 2000  
        x = mm1(n, 9, 10)
        mser5yList = mser5y(n, x)

    print('Encontrou o fim do transiente...')
    print('Eliminando as observações do estado transiente...')
    print('Tamanho de M/M/1 final:', len(x))
    pontoTruc = mser5yList * 5
    print(f'Encontrou-se o ponto de truncagem. Na posição {pontoTruc} da fila')
    #eliminando transiente
    while i < pontoTruc:
        del x[0]
        i +=1

    print('Tamanho de M/M/1 após o corte: ', len(x), '\nAgora está no estado estacionario.\n')
    n = b*m
    coletaobs = []
    i = 0
    while i < n:   
        coletaobs.append(x[i])
        i += 1
    batch = batchmeans(coletaobs, b, m)
    desPadrao = ss.stdev(batch)
    media = ss.mean(batch)
    tstudent = st.norm.interval(0.95, loc=media, scale=desPadrao) 
    print(f"Intervalo de confiança de 95% {tstudent}")


In [36]:
simulacao(10000, 9, 10)

Encontrou o fim do transiente...
Eliminando as observações do estado transiente...
Tamanho de M/M/1 final: 10000
Encontrou-se o ponto de truncagem. Na posição 310 da fila
Tamanho de M/M/1 após o corte:  9690 
Agora está no estado estacionario.

Intervalo de confiança de 95% (-0.5667376799653882, 2.4705560512577813)


In [37]:
simulacao(10000, 9.5, 10)

Ainda não se encontrou o ponto de truncagem.

A coletar mais informações até que se encontre...
Ainda não se encontrou o ponto de truncagem.

A coletar mais informações até que se encontre...
Encontrou o fim do transiente...
Eliminando as observações do estado transiente...
Tamanho de M/M/1 final: 14000
Encontrou-se o ponto de truncagem. Na posição 650 da fila
Tamanho de M/M/1 após o corte:  13350 
Agora está no estado estacionario.

Intervalo de confiança de 95% (-0.5781091071054996, 2.6323123005243922)


Com o aumento dos clientes, o tempo de espera deveria ser maior por ser igual nos dois casos, no entanto, ao observar o intervalo de confiança, com a utilização da heuristica, obtem-se um tempo proximo nos dois casos, validando que uma boa distribuição pode fazer com o que o tempo de espera na fila seja o mais proximo possivel.